# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.
import pandas as pd

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.linear_model import Ridge
#from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, log_loss
from sklearn.ensemble import BaggingRegressor

In [2]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [3]:
fires_dt.describe()

,coord_x,coord_y,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,4.669246,4.299807,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292
std,2.313778,1.229900,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,3.000000,4.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,4.000000,4.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,7.000000,5.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


# Get X and Y

Create the features data frame and target data.

In [4]:

X = fires_dt.drop( columns='area' )
Y = fires_dt['area']

X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state=0)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [42]:
# preproc1
drop_col1 = ['month','day','area']
cat_col1 = ['month','day']
num_col1 = fires_dt.drop(columns=drop_col1).columns.to_list()

preproc1 = ColumnTransformer([
    ('scalind', StandardScaler(), num_col1),
    ('categorization', OneHotEncoder(handle_unknown='infrequent_if_exist',drop='if_binary'), cat_col1)
    ], remainder='drop')


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [41]:
drop_col = ['month','day','area','ffmc','isi']
cat_col = ['month','day']
num_col = fires_dt.drop(columns=drop_col).columns.to_list()

preproc2 = ColumnTransformer([
    ('scaling', RobustScaler(), num_col),
    ('categorization', OneHotEncoder(handle_unknown='infrequent_if_exist',drop='if_binary'), cat_col),
    ('transform', PowerTransformer(method='yeo-johnson'), ['ffmc','isi'])
], remainder='drop')



## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [17]:
# Pipeline A = preproc1 + baseline
pipe_A = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', Ridge())
])


In [8]:
# Pipeline B = preproc2 + baseline
pipe_B = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', Ridge())
])

In [9]:
# Pipeline C = preproc1 + advanced model
pipe_C = Pipeline([
    ('preprocessing', preproc1),
    ('regressor', BaggingRegressor())
])

In [10]:
# Pipeline D = preproc2 + advanced model

pipe_D = Pipeline([
    ('preprocessing', preproc2),
    ('regressor', BaggingRegressor())
])  

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [51]:
scoring = ['explained_variance', 'max_error', 'neg_root_mean_squared_error', 'neg_mean_absolute_error', 'r2']

param_grid = {
    'regressor__solver':[ 'sparse_cg', 'lsqr'],
    'regressor__max_iter':[2000, 17000]
}

param_grid_advance = {
    'regressor__n_estimators':[10, 15],
    'regressor__warm_start': [False,True]
}
def get_results(pipe, grid ):
    grid_cv = GridSearchCV(
        estimator=pipe, 
        param_grid=grid, 
        scoring = scoring, 
        cv = 5,
        refit = "neg_root_mean_squared_error")
    grid_cv.fit(X_train, Y_train)

    res = grid_cv.cv_results_
    return pd.DataFrame(res)
    

In [52]:
get_results( pipe_A, param_grid  )

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__max_iter,param_regressor__solver,params,split0_test_explained_variance,split1_test_explained_variance,split2_test_explained_variance,...,std_test_neg_mean_absolute_error,rank_test_neg_mean_absolute_error,split0_test_r2,split1_test_r2,split2_test_r2,split3_test_r2,split4_test_r2,mean_test_r2,std_test_r2,rank_test_r2
0,0.009123,0.000833,0.005996,0.000632,2000,sparse_cg,"{'regressor__max_iter': 2000, 'regressor__solv...",-0.130615,-0.006434,-0.060921,...,2.167611,3,-0.167359,-0.013578,-0.060930,-0.028077,-0.143060,-0.082601,0.061719,3
1,0.007407,0.000483,0.005793,0.000396,2000,lsqr,"{'regressor__max_iter': 2000, 'regressor__solv...",-0.130515,-0.006435,-0.060900,...,2.168026,1,-0.167264,-0.013579,-0.060909,-0.028115,-0.143007,-0.082575,0.061676,1
2,0.007404,0.000489,0.005602,0.000492,17000,sparse_cg,"{'regressor__max_iter': 17000, 'regressor__sol...",-0.130615,-0.006434,-0.060921,...,2.167611,3,-0.167359,-0.013578,-0.060930,-0.028077,-0.143060,-0.082601,0.061719,3
3,0.009609,0.000518,0.006979,0.001259,17000,lsqr,"{'regressor__max_iter': 17000, 'regressor__sol...",-0.130515,-0.006435,-0.060900,...,2.168026,1,-0.167264,-0.013579,-0.060909,-0.028115,-0.143007,-0.082575,0.061676,1


In [53]:
get_results( pipe_B, param_grid )

c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__max_iter,param_regressor__solver,params,split0_test_explained_variance,split1_test_explained_variance,split2_test_explained_variance,...,std_test_neg_mean_absolute_error,rank_test_neg_mean_absolute_error,split0_test_r2,split1_test_r2,split2_test_r2,split3_test_r2,split4_test_r2,mean_test_r2,std_test_r2,rank_test_r2
0,0.015404,0.001958,0.010000,0.000632,2000,sparse_cg,"{'regressor__max_iter': 2000, 'regressor__solv...",-0.433808,0.004561,-0.256509,...,1.481334,1,-0.489445,-0.002770,-0.263235,-0.097017,-0.213632,-0.213220,0.165284,1
1,0.013307,0.000763,0.008001,0.000895,2000,lsqr,"{'regressor__max_iter': 2000, 'regressor__solv...",-0.432811,0.004495,-0.258747,...,1.487357,3,-0.488472,-0.002832,-0.265585,-0.097034,-0.213511,-0.213487,0.165087,3
2,0.017800,0.001940,0.010001,0.000632,17000,sparse_cg,"{'regressor__max_iter': 17000, 'regressor__sol...",-0.433808,0.004561,-0.256509,...,1.481334,1,-0.489445,-0.002770,-0.263235,-0.097017,-0.213632,-0.213220,0.165284,1
3,0.017712,0.001466,0.009599,0.000799,17000,lsqr,"{'regressor__max_iter': 17000, 'regressor__sol...",-0.432811,0.004495,-0.258747,...,1.487357,3,-0.488472,-0.002832,-0.265585,-0.097034,-0.213511,-0.213487,0.165087,3


In [54]:
get_results( pipe_C, param_grid_advance )

c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__n_estimators,param_regressor__warm_start,params,split0_test_explained_variance,split1_test_explained_variance,split2_test_explained_variance,...,std_test_neg_mean_absolute_error,rank_test_neg_mean_absolute_error,split0_test_r2,split1_test_r2,split2_test_r2,split3_test_r2,split4_test_r2,mean_test_r2,std_test_r2,rank_test_r2
0,0.046712,0.002248,0.009116,0.001192,10,False,"{'regressor__n_estimators': 10, 'regressor__wa...",-2.705048,-0.058183,-0.359261,...,2.037043,1,-2.902796,-0.061961,-0.393445,-0.190085,-0.874682,-0.884594,1.046282,1
1,0.047232,0.002729,0.008206,0.000398,10,True,"{'regressor__n_estimators': 10, 'regressor__wa...",-5.238171,-0.046241,-0.723106,...,2.528636,2,-5.496518,-0.049330,-0.735899,-0.234545,-0.784730,-1.460204,2.037917,4
2,0.059518,0.002297,0.009198,0.000746,15,False,"{'regressor__n_estimators': 15, 'regressor__wa...",-3.134226,-0.035275,-1.026241,...,2.681941,4,-3.317394,-0.037059,-1.081561,-0.115273,-0.626376,-1.035533,1.201686,2
3,0.063038,0.002320,0.009197,0.001162,15,True,"{'regressor__n_estimators': 15, 'regressor__wa...",-3.192717,-0.051350,-0.960845,...,3.221981,3,-3.310676,-0.055846,-1.046298,-0.086261,-0.781486,-1.056114,1.191614,3


In [47]:
get_results( pipe_D, param_grid_advance )

c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__n_estimators,param_regressor__warm_start,params,split0_test_explained_variance,split1_test_explained_variance,split2_test_explained_variance,...,std_test_neg_mean_absolute_error,rank_test_neg_mean_absolute_error,split0_test_r2,split1_test_r2,split2_test_r2,split3_test_r2,split4_test_r2,mean_test_r2,std_test_r2,rank_test_r2
0,0.053664,0.012327,0.011601,0.004029,10,False,"{'regressor__n_estimators': 10, 'regressor__wa...",-2.549349,-0.078564,-0.352278,...,2.920336,2,-2.657793,-0.079150,-0.355197,-0.202642,-0.618298,-0.782616,0.954695,3
1,0.042515,0.003619,0.008200,0.000745,10,True,"{'regressor__n_estimators': 10, 'regressor__wa...",-2.149899,-0.051306,-0.241080,...,3.323790,1,-2.197813,-0.051747,-0.242496,-0.089671,-0.963316,-0.709008,0.814177,1
2,0.059524,0.003332,0.009517,0.000448,15,False,"{'regressor__n_estimators': 15, 'regressor__wa...",-4.537360,-0.066541,-0.125364,...,2.638193,3,-4.670643,-0.067865,-0.131426,-0.139549,-0.767488,-1.155394,1.775985,4
3,0.060659,0.003587,0.009204,0.000986,15,True,"{'regressor__n_estimators': 15, 'regressor__wa...",-1.731299,-0.053615,-0.357835,...,3.273920,4,-1.806817,-0.054262,-0.363596,-0.087265,-1.254686,-0.713325,0.698201,2


# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.